# Multiple resonators on a chip, with possibility of having holes in the resonator core

Ruirements: gdspy library.
The design is generated in the following order:


   *   **Ground and contact pads**. THe idea is that you should only specify the coordinates of the contact pads; whether they should be rotated should be decided by the code itself (if the contact pad is further that 900 $\mu m$ away, it's rotated). This is done by the functions 
   
   *Generate_Pads (sample_size_along_x_axis, sample_size_along_y_axis, [d], [d1], [d2], [layer], [contact pad coordinates])*. 
   
   
   *   **TL**. Connects two contact pads from left to right. Generally, in this code everything that involves coplanar waveguides should be drawn from left to right.
   
   *Generate_TL (contact_pad_1, contact_pad2, d, d1, d2)*
   
    
   *   **Resonators** The position is specified by the left lower corner in (x,y) format. 
   
       *Generate_Resonators (initial_point_array, [circular_claw_radius], [gap], [resonator_frequency], [mode (up or down)], [layer])*.
   
   *   **DC line**. At this point the we add resonators and dc bias lines to the sample cell.
   
   *Generate_Loops* ([coordinates of the second end; the first end automatically gets connected to a contact pad], [modes], [d], [d1], [d2], [d for the contact pad], [d1 for the contact pad], [d2 for the contact pad], [array of arc coordinates], [array of arc angles], layer)
   
   
   *   **Ground grid**. Can be generated only after everything else is in place
   
   *Generate_Grid* (sample_size_along_x, sample_size_along_y, layer1, layer2)
   
 IF you want to change anything, unfortunately you'll have to do a restart.

In [1]:
!pip install gdspy

In [2]:
from design_res3 import *
from resonators3_1 import*
import gdspy
import random as rd

In [3]:
class Sample:
    
    #Содержит функции, генерирующие все площадки, TL, все джозефсоны, все петельки
    
    def __init__(self, number):
        self.name = 'sample' + str(number)
        self.cell = gdspy.Cell(self.name)
    
    def Generate_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads_cell = gdspy.Cell('Pads')
        self.Pads_cell.add(self.Pads.Generate_Ground())
        self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
        
    def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
        TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
        self.tl = TL
        self.tl_cell1 = gdspy.Cell('TL1')
        self.tl_cell2 = gdspy.Cell('TL2')
        self.tl_cell3 = gdspy.Cell('TL3')
        self.tl_cell1.add(TL.GenerateTL(d, coords = [], rots = [])) 
        self.tl_cell2.add(TL.GenerateTL(d1, coords = [], rots = [])) 
        self.tl_cell3.add(TL.GenerateTL(d2, coords = [], rots = [])) 
        #self.cell.add([gdspy.CellReference(self.tl_cell, (0, 0))])
      
    def Generate_Resonators(self, start_points, freqs, modes, ws, ss, TL_inner, TL_vacuum, TL_ground, DEs):
        self.RESONATORS1 = gdspy.Cell('RESONATORS1')
        self.RESONATORS2 = gdspy.Cell('RESONATORS2')
        self.RESONATORS3 = gdspy.Cell('RESONATORS3')
        self.RESONATORS_RESTRICTED = gdspy.Cell('RESONATORS_RESTRICTED')
        #self.resonators = []
        #self.res_cells = []
        self.res_references = []
        for i, (start, freq, mode, w, s, TLi, TLv, TLg, DE) in enumerate(zip(start_points, freqs, modes, ws, ss, TL_inner, TL_vacuum, TL_ground, DEs)):
            #self.resonators.append(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE))
            #self.res_cells.append(gdspy.Cell('res' + str(i)))
            self.RESONATORS1.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate1(mode))
            self.RESONATORS2.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate2(mode))
            self.RESONATORS3.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate3(mode))
            self.RESONATORS_RESTRICTED.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate3(mode))
            #self.res_cells[i].add(self.resonators[i].Generate3(mode))
            
        COMBINATION = gdspy.Cell('COMBINATION')
        COMBINATION.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS3, (0,0)), gdspy.CellReference(self.tl_cell3, (0,0)), 'or'))
        CUT = gdspy.Cell('CUT')
        CUT.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS2, (0,0)), gdspy.CellReference(self.tl_cell2, (0,0)), 'or'))
        CUTOUT = gdspy.Cell('CUTOUT')
        CUTOUT.add(gdspy.boolean(gdspy.CellReference(COMBINATION, (0,0)), gdspy.CellReference(CUT, (0,0)), 'not'))
        FILL = gdspy.Cell('FILL')
        FILL.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS1, (0,0)), gdspy.CellReference(self.tl_cell1, (0,0)), 'or'))
        
        CUTOUTFILL = gdspy.Cell('CUTOUTFILL')
        CUTOUTFILL.add(gdspy.boolean(gdspy.CellReference(CUTOUT, (0,0)), gdspy.CellReference(FILL, (0,0)), 'or'))
        self.cell.add(gdspy.CellReference(CUTOUTFILL, (0, 0)))

        
                        
    def Generate_Grid(self, a, b, l1, l2):
        self.grid_cell = gdspy.Cell('Grid')
        res1 = gdspy.Rectangle((400, 400), (405, b - 400), layer = l1) 
        x = 447
        while x < a - 400:
            r1 = gdspy.Rectangle((x, 400), (x + 5, b - 400), layer = l1) 
            x += 47
            res1 = gdspy.boolean(res1, r1, 'or')
            #self.grid_cell.add(r1)
        y = 400
        res2 = gdspy.Rectangle((400, 400), (400, 405), layer = l2) 
        while y < b - 400:
            r1 = gdspy.Rectangle((400, y), (a - 400, y + 5), layer = l2) 
            y += 47
            res2 = gdspy.boolean(res2, r1, 'or')
            #self.grid_cell.add(r1)
        res2 = gdspy.boolean(res2, res1, 'not')
        for area in self.Pads.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for area in self.tl.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        area = gdspy.CellReference(self.RESONATORS_RESTRICTED, (0,0))
        res1 = gdspy.boolean(res1, area, 'not', layer = l1)
        res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        
        #res = gdspy.boolean(res, self.Pads_cell.get_polygonsets(), 'not')
        self.grid_h = res1
        self.grid_v = res2
        self.grid_cell.add(res1)
        self.grid_cell.add(res2)
        self.cell.add(gdspy.CellReference(self.grid_cell, (0, 0)))
    #self.Resonators
    #self.JJ
    #self.Loops
    #self.TL

In [4]:
TL_inner = 15
TL_vacuum = 4
TL_ground = 25
DE = 14
TL_total = TL_inner + 2*(TL_vacuum + TL_ground)
number = 84
w = 4
s = 21
positions = [] 
for j in range(3):
    for i in range(14):
        if j == 0:
            positions.append(coordinates(800 + 50 + 585*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(800 + 50 + 585*i, 3900 + TL_total/2 - j*1400))
        elif j == 1:
            positions.append(coordinates(920 + 580*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(920 + 580*i, 3900 + TL_total/2 - j*1400))
        elif j == 2:
            positions.append(coordinates(950 + 585*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(950 + 585*i, 3900 + TL_total/2 - j*1400)) 
w_arr = np.full(number, w) 
s_arr = np.full(number, s) 
f_arr = np.linspace(8e9, 9e9, number) #number
rd.shuffle(f_arr)
print(f_arr)
DE_arr = np.full(number, DE)
TL_inner_arr = np.repeat(TL_inner, number)
TL_vacuum_arr = np.repeat(TL_vacuum, number)
TL_ground_arr = np.repeat(TL_ground, number)
ground_arr = np.repeat('ground', number)
updown = ['up', 'down']
updown_arr = np.tile(updown, number)

[8.01204819e+09 8.50602410e+09 8.97590361e+09 8.45783133e+09
 8.28915663e+09 8.02409639e+09 8.31325301e+09 8.40963855e+09
 8.86746988e+09 8.24096386e+09 8.25301205e+09 8.74698795e+09
 8.72289157e+09 8.14457831e+09 8.04819277e+09 8.60240964e+09
 8.93975904e+09 8.79518072e+09 8.56626506e+09 8.96385542e+09
 8.20481928e+09 8.55421687e+09 8.51807229e+09 8.83132530e+09
 8.62650602e+09 8.73493976e+09 8.90361446e+09 8.65060241e+09
 8.44578313e+09 8.84337349e+09 8.03614458e+09 8.57831325e+09
 8.21686747e+09 8.80722892e+09 8.42168675e+09 8.27710843e+09
 8.54216867e+09 8.36144578e+09 8.43373494e+09 8.49397590e+09
 8.71084337e+09 8.53012048e+09 8.33734940e+09 8.13253012e+09
 8.26506024e+09 8.39759036e+09 8.16867470e+09 8.38554217e+09
 8.87951807e+09 8.06024096e+09 8.09638554e+09 8.92771084e+09
 9.00000000e+09 8.68674699e+09 8.98795181e+09 8.30120482e+09
 8.08433735e+09 8.67469880e+09 8.66265060e+09 8.37349398e+09
 8.32530120e+09 8.12048193e+09 8.77108434e+09 8.61445783e+09
 8.63855422e+09 8.891566

In [5]:
sample = Sample(2)

sample.Generate_Pads(10000, 5000, #width, depth
                     2, #number of Pads
                     [TL_inner, TL_inner], #d
                     [TL_inner +2*TL_vacuum, TL_inner +2*TL_vacuum], #d1
                     [TL_total, TL_total], #d2
                     0, #layer
                     [coordinates(600, 2500 - TL_total/2), coordinates(9400, 2500 - TL_total/2)]) #reference coordinates for Pads
sample.Generate_TL(0, 1, TL_inner, TL_inner +2*TL_vacuum, TL_inner + 2*(TL_vacuum+TL_ground), 0)

sample.Generate_Resonators(positions, # ref coordinates
                           f_arr, #freqs
                           updown_arr, #mode
                           w_arr, #w
                           s_arr, #s
                           TL_inner_arr,
                           TL_vacuum_arr,
                           TL_ground_arr,
                           DE_arr) 


[(600, 2492.5), (673, 2492.5), (673, 3892.5), (9227, 3892.5), (9227, 2492.5), (773, 2492.5), (773, 1092.5), (9327, 1092.5), (9327, 2492.5), (9400, 2492.5)]
False PolygonSet (1 polygons, 72 vertices, layers [0], datatypes [0])
True PolygonSet (1 polygons, 72 vertices, layers [0], datatypes [0])
False PolygonSet (1 polygons, 72 vertices, layers [0], datatypes [0])
True PolygonSet (1 polygons, 72 vertices, layers [0], datatypes [0])
False PolygonSet (1 polygons, 72 vertices, layers [0], datatypes [0])
True PolygonSet (1 polygons, 72 vertices, layers [0], datatypes [0])
False PolygonSet (1 polygons, 72 vertices, layers [0], datatypes [0])
True PolygonSet (1 polygons, 72 vertices, layers [0], datatypes [0])
[(600, 2492.5), (673, 2492.5), (673, 3892.5), (9227, 3892.5), (9227, 2492.5), (773, 2492.5), (773, 1092.5), (9327, 1092.5), (9327, 2492.5), (9400, 2492.5)]
False PolygonSet (1 polygons, 76 vertices, layers [0], datatypes [0])
True PolygonSet (1 polygons, 76 vertices, layers [0], datatype

coupling length: 383.40158190004894
x coordinate of end of resonator 5459.801581900049
DE= 14
...
coupling length: 383.40158190004894
x coordinate of end of resonator 5459.801581900049
DE= 14
...
coupling length: 368.9335976774056
x coordinate of end of resonator 6030.333597677406
DE= 14
...
coupling length: 368.9335976774056
x coordinate of end of resonator 6030.333597677406
DE= 14
...
coupling length: 368.9335976774056
x coordinate of end of resonator 6030.333597677406
DE= 14
...
coupling length: 368.9335976774056
x coordinate of end of resonator 6030.333597677406
DE= 14
...
coupling length: 374.9982595570342
x coordinate of end of resonator 6036.398259557034
DE= 14
...
coupling length: 374.9982595570342
x coordinate of end of resonator 6036.398259557034
DE= 14
...
coupling length: 374.9982595570342
x coordinate of end of resonator 6036.398259557034
DE= 14
...
coupling length: 374.9982595570342
x coordinate of end of resonator 6036.398259557034
DE= 14
...
coupling length: 385.0193100

coupling length: 394.45061884241346
x coordinate of end of resonator 3765.8506188424135
DE= 14
...
coupling length: 394.45061884241346
x coordinate of end of resonator 3765.8506188424135
DE= 14
...
coupling length: 391.06961353804985
x coordinate of end of resonator 4342.46961353805
DE= 14
...
coupling length: 391.06961353804985
x coordinate of end of resonator 4342.46961353805
DE= 14
...
coupling length: 391.06961353804985
x coordinate of end of resonator 4342.46961353805
DE= 14
...
coupling length: 391.06961353804985
x coordinate of end of resonator 4342.46961353805
DE= 14
...
coupling length: 388.29607017962394
x coordinate of end of resonator 4339.696070179624
DE= 14
...
coupling length: 388.29607017962394
x coordinate of end of resonator 4339.696070179624
DE= 14
...
coupling length: 388.29607017962394
x coordinate of end of resonator 4339.696070179624
DE= 14
...
coupling length: 388.29607017962394
x coordinate of end of resonator 4339.696070179624
DE= 14
...
coupling length: 378.6

coupling length: 396.16313961886385
x coordinate of end of resonator 2647.563139618864
DE= 14
...
coupling length: 396.16313961886385
x coordinate of end of resonator 2647.563139618864
DE= 14
...
coupling length: 406.15538498017054
x coordinate of end of resonator 2657.555384980171
DE= 14
...
coupling length: 406.15538498017054
x coordinate of end of resonator 2657.555384980171
DE= 14
...
coupling length: 406.15538498017054
x coordinate of end of resonator 2657.555384980171
DE= 14
...
coupling length: 406.15538498017054
x coordinate of end of resonator 2657.555384980171
DE= 14
...
coupling length: 376.0284745558172
x coordinate of end of resonator 3212.4284745558175
DE= 14
...
coupling length: 376.0284745558172
x coordinate of end of resonator 3212.4284745558175
DE= 14
...
coupling length: 376.0284745558172
x coordinate of end of resonator 3212.4284745558175
DE= 14
...
coupling length: 376.0284745558172
x coordinate of end of resonator 3212.4284745558175
DE= 14
...
coupling length: 382

coupling length: 373.9736741484084
x coordinate of end of resonator 9060.373674148408
DE= 14
...
coupling length: 373.9736741484084
x coordinate of end of resonator 9060.373674148408
DE= 14
...
coupling length: 373.9736741484084
x coordinate of end of resonator 9060.373674148408
DE= 14
...
coupling length: 373.9736741484084
x coordinate of end of resonator 9060.373674148408
DE= 14
...
coupling length: 404.3555826833603
x coordinate of end of resonator 9090.75558268336
DE= 14
...
coupling length: 404.3555826833603
x coordinate of end of resonator 9090.75558268336
DE= 14
...
coupling length: 404.3555826833603
x coordinate of end of resonator 9090.75558268336
DE= 14
...
coupling length: 404.3555826833603
x coordinate of end of resonator 9090.75558268336
DE= 14
...


In [6]:
#gdspy.LayoutViewer(depth = 2)

In [7]:
sample.Generate_Grid(10000,5000,0,0)

In [8]:
gdspy.write_gds('resonators3_1_update.gds', cells=None, name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)